<a href="https://colab.research.google.com/github/MarkZuck10/economic-recession-prediction/blob/main/VADERembeddings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 1. Importing Dataset

In [6]:
import pandas as pd

df = pd.read_csv('drive/MyDrive/01-02-23_VADER-sentiment.csv')
df.shape

(21964, 3)

In [8]:
df.isna().sum().sort_values(ascending=False)

,0
tweetid,10
vader_sentiment,10
text,0


In [9]:
df = df.dropna()

In [10]:
df.isna().sum().sort_values(ascending=False)

,0
text,0
tweetid,0
vader_sentiment,0


In [7]:
print(df.duplicated().sum())

2


In [11]:
df = df.drop_duplicates()

In [12]:
print(df.duplicated().sum())

0


In [13]:
df.shape

(21954, 3)

In [14]:
sentiment_counts = df['vader_sentiment'].value_counts()

print(sentiment_counts)

vader_sentiment
Negative    9810
Positive    7742
Neutral     4402
Name: count, dtype: int64


# 2. generating Embeddings

In [ ]:
from transformers import BertTokenizer, BertModel
import torch

# Loading pre-trained BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

In [2]:
def process_batch(texts_batch):
    inputs = tokenizer(texts_batch, return_tensors='pt', padding=True, truncation=True, max_length=128)
    with torch.no_grad():
        outputs = model(**inputs)
        return outputs.last_hidden_state.mean(dim=1).cpu().numpy()

In [27]:
texts = df['text'].tolist()

batch_size = 256
embeddings = []

In [28]:
for i in range(0, len(texts), batch_size):
    batch_texts = texts[i:i + batch_size]
    batch_embeddings = process_batch(batch_texts)
    embeddings.append(batch_embeddings)

In [29]:
# Combining all embeddings
import numpy as np
embeddings_np = np.vstack(embeddings)

embeddings_np.shape

(21954, 768)

# 3. Saving embeddings

In [30]:
import numpy as np

np.save("/content/drive/MyDrive/01-02-23-embeddings.npy", embeddings_np)